In [ ]:
test__raw(D_raw,hmms)
test__norm(D_norm,hmms)

In [ ]:
%%time
##################################################################
##################################################################
############### Append information to pairwise hits   ############
######## Calculate hierarchial information for histogram #########
##################################################################
##################################################################

# using_coo_izip
D_curr = D_raw
# D_curr = D_norm

xdata = []
ydata = []
vdata = []
sfbools = []
topobools = []
archbools = []
classbools = []

for x,y,v in using_tocoo_izip(D_curr):
    xdata.append(x)
    ydata.append(y)
    vdata.append(v)
    
    sameclass = class_db[x] == class_db[y]
    if sameclass:
        samearch = arch_db[x] == arch_db[y]
    else:
        samearch = False        
    if samearch:
        sametopo = topo_db[x] == topo_db[y]
    else:
        sametopo = False    
    if sametopo:
        samesf   = sf_db[x] == sf_db[y]
    else:
        samesf   = False

    classbools.append ( sameclass )
    archbools. append ( samearch )
    topobools. append ( sametopo )
    sfbools.   append ( samesf )


In [ ]:
import matplotlib
%matplotlib inline
matplotlib.use("Agg")
from matplotlib import pyplot as plt
from scipy import stats
# from mpl_toolkits.mplot3d import Axes3D
# kwargs = {
#     'projection':'3d',
# }


vals0 = np.array(vdata)


kwargs = {'ylabel':'count',
          'xlabel':'noramlised ISS score',
         'title':'CATH_S100_V410',
         }
plt.close()
siz = [450,400]
DPI=100.
fig = plt.figure(figsize = [x/float(DPI) for x in siz], dpi=DPI)
ax1 = fig.add_subplot(111,**kwargs)
ax1b = ax1.twinx()

# ax1.set_yscale('log')
# ax1.set_xscale('log')
# vals = np.maximum(0,vdata)
# lbls = [None]



lbl = 'All pairwise'
kargs = {'label':lbl}


vals = np.ma.array(vdata,mask = None)

ys,binEdges = np.histogram(vals,bins=50, 
                          density = True
                          )
bincenters  = 0.5*( binEdges[1:] + binEdges[:-1] )

ax1.plot( bincenters, ys, '-' ,

# binsA1 = ax1.hist( vals,100, 
#     facecolor = 'b',

# edgecolor = 'r',facecolor = 'r',
#     marker ='o',
    # c = colors,
#      s =100,
     alpha = 0.2,
     **kargs
#     label = ''
     )



lbl = 'Same Class ( 0.5% )'
kargs = {'label': lbl}
vals = np.array(vdata)[ listANDNOT(classbools,archbools)]

ys,binEdges = np.histogram(vals,bins=50, 
                          density = True
                          )
bincenters  = 0.5 * ( binEdges[1:] + binEdges[:-1] )

ax1b.plot( bincenters, ys, '-' ,
# binsB1 = ax1b.hist( vals,100, 
#     facecolor = 'g',
     alpha = 0.4,
     **kargs
     )





lbl = 'Same Architecture ( 0.5% )'
kargs = {'label': lbl}
vals = np.array(vdata)[listANDNOT(archbools, topobools)]


ys,binEdges = np.histogram(vals,bins=50,
                          density = True
                          )
bincenters  = 0.5*( binEdges[1:] + binEdges[:-1] )

ax1b.plot( bincenters, ys, '-' ,

# binsB1 = ax1b.hist( vals,100, 
#     facecolor = 'g',

    # edgecolor = 'r',facecolor = 'r',
#     marker ='o',
    # c = colors,
#      s =100,
     alpha = 0.5,
     **kargs
     )




lbl = 'Same Topology ( 0.5% )'
kargs = { 'label': lbl }
vals = np.array(vdata)[listANDNOT(topobools,sfbools)]

ys,binEdges = np.histogram(vals,bins=50, 
                          density = True
                          )
bincenters  = 0.5*( binEdges[1:] + binEdges[:-1] )

ax1b.plot( bincenters, ys, '-' ,

# binsB1 = ax1b.hist( vals,100, 
#     facecolor = 'g',
    # edgecolor = 'r',facecolor = 'r',
#     marker ='o',
    # c = colors,
#      s =100,
     alpha = 0.6,
     **kargs
     )






lbl = 'Same superfamily ( 0.5% )'
kargs = {'label': lbl}
vals = np.array(vdata)[sfbools]

ys,binEdges = np.histogram(vals,bins=50, 
                          density = True
                          )
bincenters  = 0.5*( binEdges[1:] + binEdges[:-1] )

ax1b.plot( bincenters, ys, '-' ,

# binsB1 = ax1b.hist( vals,100, 
#     facecolor = 'r',
         
    # edgecolor = 'r',facecolor = 'r',
#     marker ='o',
    # c = colors,
#      s =100,
     alpha = 0.4,
     **kargs
     )






lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax1b.get_legend_handles_labels()

ax1.legend(lines + lines2,
           labels + labels2,
           loc = 2)
ax1b.set_ylim( [-.2, 1] )



# # vals = np
# ax1.legend( )
# ax1b.legend( )
# ax1.legend( loc = 1)
# ax1b.legend(loc = 2 )
# plt.legend()

In [ ]:
import matplotlib
%matplotlib inline
matplotlib.use("Agg")
from matplotlib import pyplot as plt
# from mpl_toolkits.mplot3d import Axes3D
# kwargs = {
#     'projection':'3d',
# }



kwargs = {'ylabel':'count',
          'xlabel':'noramlised ISS score',
         'title':'CATH_S100_V410',
         }
plt.close('all')
siz = [450,400]
DPI=100.
fig = plt.figure(figsize = [x/float(DPI) for x in siz], dpi=DPI)
ax1 = fig.add_subplot(111,**kwargs)
ax1b = ax1.twinx()
# ax1.set_yscale('log')
# ax1.set_xscale('log')

# vals = np.maximum(0,vdata)
# lbls = [None]

lbl = 'All pairwise'
kargs = {'label':lbl}

vals = np.ma.array(vdata,mask = None)

binsA1 = ax1.hist( vals,50, 
    edgecolor = 'b',facecolor = 'b',
    # edgecolor = 'r',facecolor = 'r',
#     marker ='o',
    # c = colors,
#      s =100,
     alpha = 0.3,
     **kargs
#     label = ''
     )



lbl = 'Excluded same superfamily ( 99.5% )'
# vals = np.delete(np.array(vdata), (sfbools) )
vals = np.array(vdata)[listNOT(sfbools)]


kargs = {'label': lbl}

# vals = np.array(vdata)[sfbools]

binsB1 = ax1b.hist( vals,50, 
    edgecolor = 'r',facecolor = 'r',
    # edgecolor = 'r',facecolor = 'r',
#     marker ='o',
    # c = colors,
#      s =100,
     alpha = 0.3,
     **kargs
     )
lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax1b.get_legend_handles_labels()

ax1.legend(lines + lines2,
           labels + labels2,
           loc =0)
# ax1.legend( )
# ax1b.legend( )
# ax1.legend( loc = 1)
# ax1b.legend(loc = 2 )
# plt.legend()